In [ ]:
from bs4 import BeautifulSoup
import requests
import os

### 정부 사업

In [ ]:
page_num = 1
url = f"https://www.msit.go.kr/bbs/list.do?sCode=user&mId=129&mPid=224&pageIndex={page_num}&bbsSeqNo=100&nttSeqNo=&searchOpt=ALL&searchTxt="
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

# 페이지에서 첫 번째 h1 태그 찾기
div = soup.find_all('div',class_="board-link", limit=1)  